In [ ]:
import networkx as nx
import numpy as np
import pandas as pd
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt
import urllib.request as urlrequest
from IPython.display import Image
from collections import Counter
from goose3 import Goose
from goose3.network import NetworkError
from bertopic import BERTopic
from scipy.cluster import hierarchy as sch
from sklearn.feature_extraction.text import CountVectorizer
from urllib.parse import urlparse
from scipy.spatial.distance import pdist, squareform


In [ ]:
pd.reset_option("all")

In [118]:
pd.set_option('expand_frame_repr', True)
pd.set_option("display.max_columns", None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [86]:
#d_200908 = pd.read_csv('200908.CSV',sep='\t',header=None)
d_20140523 = pd.read_csv('20140523.export.CSV',sep='\t',header=None)
d_20140524 = pd.read_csv('20140524.export.CSV',sep='\t',header=None)
##elliot rodgers CA
d_20160612 = pd.read_csv('20160612.export.CSV',sep='\t',header=None)
##orlando night club FL
d_20170812 = pd.read_csv('20170812.export.CSV',sep='\t',header=None)
##charlottesville VA
d_20171031 = pd.read_csv('20171031.export.CSV',sep='\t',header=None)
##truck attack NY
d_20180423 = pd.read_csv('20180423.export.CSV',sep='\t',header=None)
##TORONTO skip for now
d_20200526 = pd.read_csv('20200526.export.CSV',sep='\t',header=None)
##george floyd MN
d_20250101 = pd.read_csv('20250101.export.CSV',sep='\t',header=None)
##truck attack neworleans TN
d_20250602 = pd.read_csv('20250602.export.CSV',sep='\t',header=None)
#anti israel CO


/Users/emilylugos/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (8,9,10,11,12,13,14,19) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/emilylugos/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (8,9,11,14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/emilylugos/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (8,11,14,19,24) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/emilylugos/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (26,27,28) have mixed types.Specify dtype option on import or set low_memory=False.
  exe

In [3]:
map = {
    0: "GlobalEventID", 
    1: "date", 
    2: "MonthYear", 
    3: "Year", 
    4: "FractionDate", 
    5: "Actor1Code", 
    6: "Actor1Name", 
    7: "Actor1CountryCode", 
    8: "Actor1KnownGroupCode", 
    9: "Actor1EthnicCode",
    10: "Actor1Religion1Code", 
    11: "Actor1Religion2Code", 
    12: "Actor1Type1Code", 
    13: "Actor1Type2Code",
    14: "Actor1Type3Code", 
    15: "Actor2Code", 
    16: "Actor2Name", 
    17: "Actor2CountryCode", 
    18: "Actor2KnownGroupCode", 
    19: "Actor2EthnicCode",
    20: "Actor2Religion1Code", 
    21: "Actor2Religion2Code", 
    22: "Actor2Type1Code", 
    23: "Actor2Type2Code", 
    24: "Actor2Type3Code", 
    25: "IsRootEvent", 
    26: "EventCode", 
    27: "EventBaseCode", 
    28: "EventRootCode", 
    29: "QuadClass",
    30: "GoldsteinScale", 
    31: "NumMentions", 
    32: "NumSources", 
    33: "NumArticles", 
    34: "AvgTone", 
    35: "Actor1Geo_Type", 
    36: "Actor1Geo_Fullname", 
    37: "Actor1Geo_CountryCode", 
    38: "Actor1Geo_ADM1Code", 
    39: "Actor1Geo_Lat",
    40: "Actor1Geo_Long", 
    41: "Actor1Geo_FeatureID", 
    42: "Actor2Geo_Type", 
    43: "Actor2Geo_Fullname", 
    44: "Actor2Geo_CountryCode", 
    45: "Actor2Geo_ADM1Code", 
    46: "Actor2Geo_Lat",
    47: "Actor2Geo_Long",
    48: "Actor2Geo_FeatureID", 
    49: "ActionGeo_Type", 
    50: "ActionGeo_Fullname", 
    51: "ActionGeo_CountryCode", 
    52: "ActionGeo_ADM1Code", 
    53: "ActionGeo_Lat", 
    54: "ActionGeo_Long", 
    55: "ActionGeo_FeatureID",
    56: "DATEADDED", 
    57: "SOURCEURL"
}


In [4]:

def clean(df):
    values_to_keep = [193,190,10,12,173,90,112,145,20,40,111,43,42,51,141]
    df = df.rename(columns=map)
    df = df[(df['Actor1Geo_CountryCode'] == 'US') | (df['Actor2Geo_CountryCode'] == 'US') | (df['ActionGeo_CountryCode'] == 'US')]
    df = df[df['EventCode'].isin(values_to_keep)]
    df = df.drop(columns=['MonthYear', 'FractionDate','Actor1Code', 'Actor1Name','Actor1CountryCode', 'Actor1KnownGroupCode','Actor1EthnicCode', 'Actor1Religion1Code','Actor1Religion2Code', 'Actor1Type1Code','Actor1Type2Code','Actor1Type3Code','Actor2Code', 'Actor2Name','Actor2CountryCode', 'Actor2KnownGroupCode','Actor2EthnicCode', 'Actor2Religion1Code','Actor2Religion2Code', 'Actor2Type1Code','Actor2Type2Code','Actor2Type3Code','Actor1Geo_Type', 'Actor1Geo_Fullname','Actor1Geo_ADM1Code', 'Actor1Geo_Lat','Actor1Geo_Long', 'Actor1Geo_FeatureID', 'Actor2Geo_Type', 'Actor2Geo_Fullname', 'Actor2Geo_ADM1Code', 'Actor2Geo_Lat','Actor2Geo_Long','Actor2Geo_FeatureID'], axis=1)
    df = df[(df['AvgTone'] < 1)]
    return df

def simplify(domain):
    parts = domain.split('.')
    # Remove common subdomain prefixes like 'www'
    if parts[0] == 'www':
        parts = parts[1:]
    # Heuristic: use the first part before the main TLD (.com, .org, etc.)
    return parts[-3] if len(parts) > 2 else parts[0]

In [5]:
d_20140523 = clean(d_20140523)
d_20160612 = clean(d_20160612)
d_20170812 = clean(d_20170812)
d_20171031 = clean(d_20171031)
d_20180423 = clean(d_20180423)
d_20200526 = clean(d_20200526)
d_20250101 = clean(d_20250101)
d_20250602 = clean(d_20250602)
d_20140524 = clean(d_20140524)


NameError: name 'd_20140523' is not defined

In [89]:
d_20140523_CA = d_20140523[(d_20140523['ActionGeo_FeatureID'] == 'CA')]
#102
d_20140524_CA= d_20140524[(d_20140524['ActionGeo_FeatureID'] == 'CA')]
#309
d_20160612_FL= d_20160612[(d_20160612['ActionGeo_FeatureID'] == 'FL')]
#2349
d_20170812_VA= d_20170812[(d_20170812['ActionGeo_FeatureID'] == 'VA')]
#1037
d_20171031_NY= d_20171031[(d_20171031['ActionGeo_FeatureID'] == 'NY')]
#2873 most likely half
#d_20180423= d_20180423[(d_20180423['ActionGeo_FeatureID'] == 'FL')]
d_20200526_MN = d_20200526[(d_20200526['ActionGeo_FeatureID'] == 'MN')]
#405 most likely more in coming days
d_20250101_TN = d_20250101[(d_20250101['ActionGeo_FeatureID'] == 'TN')]
##not viable
d_20250602_CO = d_20250602[(d_20250602['ActionGeo_FeatureID'] == 'CO')]
#1170

In [5]:
def goose_it(df):
    for i in df.index:
        try:
            url = df.loc[i, 'SOURCEURL']
            article = g.extract(url=url)
            df.at[i, 'domain'] = article.domain
            df.at[i,'target'] = simplify(article.domain)
            df.at[i, 'title'] = article.title
            df.at[i, 'text'] = article.cleaned_text
            df.at[i, 'description'] = article.opengraph.get('description', '')
            
        except NetworkError:
            print(f"NetworkError: Dropping index {i} with URL: {df.loc[i, 'SOURCEURL']}")
            df.drop(index=i, inplace=True)
        except Exception as e:
            print(f"Error at index {i}: {e}")
            print(df.loc[i])
            df.drop(index=i, inplace=True)
    return df

In [90]:
f_20140524_CA = goose_it(d_20140524_CA)
f_20160612_FL = goose_it(d_20160612_FL)
f_20170812_VA = goose_it(d_20170812_VA)
f_20171031_NY = goose_it(d_20171031_NY)
f_20250602_CO = goose_it(d_20250602_CO)

/Users/emilylugos/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/Users/emilylugos/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


NetworkError: Dropping index 3872 with URL: http://www.wtae.com/national/Rampage-in-college-town-began-with-stabbings/26153616?absolute=true/RK=0


/Users/emilylugos/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


NetworkError: Dropping index 4027 with URL: http://www.foxnews.com/us/2014/05/24/calif-gunman-stabbed-3-at-home-before-shooting-rampage//RK=0
NetworkError: Dropping index 4032 with URL: http://www.foxnews.com/us/2014/05/24/calif-gunman-stabbed-3-at-home-before-shooting-rampage//RK=0
NetworkError: Dropping index 4543 with URL: http://www.businessinsider.com/santa-barbara-shooting-details-2014-5/RK=0
NetworkError: Dropping index 4544 with URL: http://www.businessinsider.com/santa-barbara-shooting-details-2014-5/RK=0
Error at index 4814: HTTPConnectionPool(host='devel.malaysiandigest.com', port=80): Max retries exceeded with url: /frontpage/29-4-tile/502216-us-police-confirm-drive-by-killer-is-elliot-rodger-son-of-hunger-games-asst-director.html (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fd00a32f940>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
GlobalEventID                                             

Parser lxml failed to parse the content
Parser soup failed to parse the content


...

NetworkError: Dropping index 106732 with URL: https://www.israelnationalnews.com/news/409341


Publish date Jun 2, 2025, 8:43 PM (GMT+3) could not be resolved to UTC
Publish date Jun 2, 2025, 8:43 PM (GMT+3) could not be resolved to UTC


NetworkError: Dropping index 107112 with URL: https://www.israelnationalnews.com/news/409331
NetworkError: Dropping index 107127 with URL: https://www.israelnationalnews.com/news/409331
NetworkError: Dropping index 107211 with URL: https://www.israelnationalnews.com/news/409331
NetworkError: Dropping index 107214 with URL: https://www.marketscreener.com/quote/currency/EURO-ISRAELI-NEW-SHEKEL-E-60037304/news/Colorado-attack-suspect-charged-with-federal-hate-crime-50133138/
NetworkError: Dropping index 107227 with URL: https://www.israelnationalnews.com/news/409341
NetworkError: Dropping index 107497 with URL: https://www.marketscreener.com/quote/currency/EURO-ISRAELI-NEW-SHEKEL-E-60037304/news/Colorado-attack-suspect-charged-with-federal-hate-crime-50133138/
NetworkError: Dropping index 107630 with URL: https://www.marketscreener.com/quote/currency/EURO-ISRAELI-NEW-SHEKEL-E-60037304/news/Colorado-attack-suspect-charged-with-federal-hate-crime-50133138/


Publish date Jun 2, 2025, 8:43 PM (GMT+3) could not be resolved to UTC


NetworkError: Dropping index 116094 with URL: https://nationalpost.com/news/canada/canadian-jewish-organizations-condemn-colorado-attack-call-for-more-action-at-home
NetworkError: Dropping index 116095 with URL: https://nationalpost.com/news/canada/canadian-jewish-organizations-condemn-colorado-attack-call-for-more-action-at-home
NetworkError: Dropping index 116103 with URL: https://nationalpost.com/news/canada/canadian-jewish-organizations-condemn-colorado-attack-call-for-more-action-at-home
NetworkError: Dropping index 121967 with URL: https://www.wvlkam.com/news/toensing-to-newsmax-colo-attack-is-a-federal-case/
NetworkError: Dropping index 122321 with URL: https://www.wvlkam.com/news/toensing-to-newsmax-colo-attack-is-a-federal-case/
NetworkError: Dropping index 122328 with URL: https://www.wvlkam.com/news/toensing-to-newsmax-colo-attack-is-a-federal-case/
NetworkError: Dropping index 122342 with URL: https://www.wvlkam.com/news/toensing-to-newsmax-colo-attack-is-a-federal-case/
Ne

In [106]:
f_20140524_CA.to_csv('f_20140524_CA.csv',index=False)

In [105]:
f_20160612_FL.to_csv('f_20160612_FL.csv',index=False)

In [ ]:
f_20170812_VA.to_csv('f_20170812_VA.csv',index=False)

In [103]:
f_20171031_NY.to_csv('f_20171031_NY.csv', index=False)

In [102]:
f_20250602_CO.to_csv('f_20250602_CO.csv', index=False)

In [7]:
mbfcs = pd.read_json('mbfc_sources.jsonl', lines=True)
print(mbfcs.head()) # Display the first few rows of the DataFrame

     display_name            name  \
0  Accountable.US  accountable_us   
1          Act.TV          act_tv   
2  Action Network  action_network   
3     Aftonbladet     aftonbladet   
4   Alaska Beacon   alaska_beacon   

                                              mbfc_url country  \
0       https://mediabiasfactcheck.com/accountable-us/     USA   
1               https://mediabiasfactcheck.com/act-tv/     USA   
2  https://mediabiasfactcheck.com/action-network-bias/     USA   
3          https://mediabiasfactcheck.com/aftonbladet/  Sweden   
4   https://mediabiasfactcheck.com/alaska-beacon-bias/     USA   

       factuality press_freedom                      website  \
0  MOSTLY FACTUAL          None  https://www.accountable.us/   
1  MOSTLY FACTUAL          None                http://act.tv   
2  MOSTLY FACTUAL          None   https://actionnetwork.org/   
3           MIXED          None   https://www.aftonbladet.se   
4            HIGH          None    https://alaskabeacon.com/

In [4]:
def replace_nans(text_list, replacement=""):
    return ["" if str(x).strip().lower() == "nan" or x != x else x for x in text_list]

def target(df, mbfcs):
    df['factuality'] = ''  # Initialize column as object type
    df['factuality'] = df['factuality'].astype('object')
    # Normalize mbfcs keys for matching
    mbfcs['normalized_display_name'] = mbfcs['display_name'].str.lower().str.replace(" ", "", regex=False)
    mbfcs['normalized_website'] = mbfcs['website'].apply(lambda x: urlparse(x).hostname if pd.notnull(x) else '').str.lower()

    for i in df.index:
        target_clean = str(df.loc[i, 'target']).lower().replace(" ", "")
        domain_clean = str(df.loc[i, 'domain']).lower().replace(" ", "")

        match = mbfcs[mbfcs['normalized_display_name'] == target_clean]
        if not match.empty:
            df.at[i, 'factuality'] = match.iloc[0]['factuality']
            continue

        match = mbfcs[mbfcs['normalized_website'] == domain_clean]
        if not match.empty:
            df.at[i, 'factuality'] = match.iloc[0]['factuality']
        else:
            df.at[i, 'factuality'] = None

    return df

In [5]:
f_20171031_NY = pd.read_csv('f_20171031_NY.csv')
f_20170812_VA = pd.read_csv('f_20170812_VA.csv')
f_20160612_FL = pd.read_csv('f_20160612_FL.csv')
f_20140524_CA = pd.read_csv('f_20140524_CA.csv')
f_20250602_CO = pd.read_csv('f_20250602_CO.csv')

In [8]:
f_20171031_NY = target(f_20171031_NY, mbfcs)
f_20170812_VA = target(f_20170812_VA, mbfcs)
f_20160612_FL = target(f_20160612_FL, mbfcs)
f_20250602_CO = target(f_20250602_CO, mbfcs)
f_20140524_CA = target(f_20140524_CA, mbfcs)

In [9]:
result_series = set(f_20140524_CA.loc[f_20140524_CA['factuality'].isnull(), 'domain']) | \
                set(f_20171031_NY.loc[f_20171031_NY['factuality'].isnull(), 'domain'])| \
                set(f_20170812_VA.loc[f_20170812_VA['factuality'].isnull(), 'domain'])| \
                set(f_20160612_FL.loc[f_20160612_FL['factuality'].isnull(), 'domain'])| \
                set(f_20250602_CO.loc[f_20250602_CO['factuality'].isnull(), 'domain'])


In [10]:
result_series

{'101touchfm.co.uk',
 '1055online.iheart.com',
 '1360kktx.iheart.com',
 '590kid.com',
 '790talknow.com',
 '937now.iheart.com',
 '95rockfm.com',
 '973thegame.iheart.com',
 '991thewhale.com',
 '999thepoint.com',
 'adage.com',
 'afr.net',
 'appleinsider.com',
 'archive.thinkprogress.org',
 'archiwum.thenews.pl',
 'article.wn.com',
 'au.news.yahoo.com',
 'augustafreepress.com',
 'axisoflogic.com',
 'bgr.com',
 'biztoc.com',
 'blog.timesunion.com',
 'bloody-disgusting.com',
 'bob949.iheart.com',
 'bonhamjournal.com',
 'brobible.com',
 'brooklyneagle.com',
 'brudirect.com',
 'bwog.com',
 'catholicphilly.com',
 'cbn.com',
 'cinemablaze.com',
 'collider.com',
 'collive.com',
 'colombogazette.com',
 'columbiaspectator.com',
 'commercialobserver.com',
 'contacto-latino.com',
 'cyprusbusinessmail.com',
 'dailythepatriot.com',
 'dailytimes.com.pk',
 'dailyvoice.com',
 'datechguyblog.com',
 'dnd.com.pk',
 'dominicanrepublicpost.com',
 'dunyanews.tv',
 'ecommerce-journal.com',
 'efe.com',
 'en.aswat

In [11]:
vectorizer_model = CountVectorizer(stop_words="english")
tm_20250602_CO = BERTopic(vectorizer_model=vectorizer_model)
tm_20140524_CA = BERTopic(vectorizer_model=vectorizer_model)
tm_20171031_NY = BERTopic(vectorizer_model=vectorizer_model)
tm_20170812_VA = BERTopic(vectorizer_model=vectorizer_model)
tm_20160612_FL = BERTopic(vectorizer_model=vectorizer_model)

In [12]:
l_20160612_FL = f_20160612_FL['text'].tolist()
l_20160612_FL = replace_nans(l_20160612_FL)
l_20250602_CO = f_20250602_CO['text'].tolist()
l_20250602_CO = replace_nans(l_20250602_CO)
l_20140524_CA = f_20140524_CA['text'].tolist()
l_20140524_CA = replace_nans(l_20140524_CA)
l_20171031_NY = f_20171031_NY['text'].tolist()
l_20171031_NY = replace_nans(l_20171031_NY)
l_20170812_VA = f_20170812_VA['text'].tolist()
l_20170812_VA = replace_nans(l_20170812_VA)

In [18]:
tm_20250602_CO_topics, tm_20250602_CO_probs = tm_20250602_CO.fit_transform(l_20250602_CO)
tm_20160612_FL_topics, tm_20160612_FL_probs = tm_20160612_FL.fit_transform(l_20160612_FL)
tm_20140524_CA_topics, tm_20140524_CA_probs = tm_20140524_CA.fit_transform(l_20140524_CA)
tm_20171031_NY_topics, tm_20171031_NY_probs = tm_20171031_NY.fit_transform(l_20171031_NY)
tm_20170812_VA_topics, tm_20170812_VA_probs = tm_20170812_VA.fit_transform(l_20170812_VA)



In [ ]:
topic_model = BERTopic()
topics, probs = topic_model.fit_transform(l_20250602_CO)
hierarchical_topics = topic_model.hierarchical_topics(l_20250602_CO)
vectorizer_model = CountVectorizer(stop_words="english")
tm_20250602_CO = BERTopic(vectorizer_model=vectorizer_model)
tm_20250602_CO_topics, tm_20250602_CO_probs = tm_20250602_CO.fit_transform(l_20250602_CO)


100%|██████████| 30/30 [00:00<00:00, 320.78it/s]


In [ ]:
# Define custom vectorizer to remove stop words
vectorizer_model = CountVectorizer(stop_words="english")
# Initialize BERTopic with custom vectorizer
tm_20250602_CO = BERTopic(vectorizer_model=vectorizer_model)
# Fit the model on your documents
tm_20250602_CO_topics, tm_20250602_CO_probs = tm_20250602_CO.fit_transform(l_20250602_CO)
# Use the same model instance to generate hierarchical topics
tm_20250602_CO_htopics = tm_20250602_CO.hierarchical_topics(l_20250602_CO)

# Initialize BERTopic with custom vectorizer
tm_20160612_FL = BERTopic(vectorizer_model=vectorizer_model)
# Fit the model on your documents
tm_20160612_FL_topics, tm_20160612_FL_probs = tm_20160612_FL.fit_transform(l_20160612_FL)
# Use the same model instance to generate hierarchical topics
tm_20160612_FL_htopics = tm_20160612_FL.hierarchical_topics(l_20160612_FL)

# Initialize BERTopic with custom vectorizer
tm_20140524_CA = BERTopic(vectorizer_model=vectorizer_model)
# Fit the model on your documents
tm_20140524_CA_topics, tm_20140524_CA_probs = tm_20140524_CA.fit_transform(l_20140524_CA)
# Use the same model instance to generate hierarchical topics
tm_20140524_CA_htopics = tm_20140524_CA.hierarchical_topics(l_20140524_CA)

# Initialize BERTopic with custom vectorizer
tm_20171031_NY = BERTopic(vectorizer_model=vectorizer_model)
# Fit the model on your documents
tm_20171031_NY_topics, tm_20171031_NY_probs = tm_20171031_NY.fit_transform(l_20171031_NY)
# Use the same model instance to generate hierarchical topics
tm_20171031_NY_htopics = tm_20171031_NY.hierarchical_topics(l_20171031_NY)

# Initialize BERTopic with custom vectorizer
tm_20170812_VA = BERTopic(vectorizer_model=vectorizer_model)
# Fit the model on your documents
tm_20170812_VA_topics, tm_20170812_VA_probs = tm_20170812_VA.fit_transform(l_20170812_VA)
# Use the same model instance to generate hierarchical topics
tm_20170812_VA_htopics = tm_20170812_VA.hierarchical_topics(l_20170812_VA)


100%|██████████| 9/9 [00:00<00:00, 171.34it/s]


In [113]:
tm_20250602_CO_htopics[['Parent_ID','Topics','Child_Left_ID','Child_Right_ID','Distance']]

,Parent_ID,Topics,Child_Left_ID,Child_Right_ID,Distance
32,66,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",63,65,1.450902
31,65,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14,...",64,62,1.088056
30,64,"[0, 1, 2, 3, 4, 5, 7, 12, 14, 15, 17, 19, 23, ...",43,60,1.041470
29,63,"[10, 18, 25, 31]",61,25,0.958779
28,62,"[6, 8, 9, 11, 13, 16, 20, 21, 22, 24, 27, 28, ...",57,29,0.958369
27,61,"[10, 18, 31]",31,59,0.921659
26,60,"[0, 1, 2, 3, 4, 5, 7, 12, 14, 15, 17, 26, 30, 33]",53,58,0.916831
25,59,"[10, 18]",18,10,0.874774
24,58,"[4, 5, 12, 30]",54,30,0.820557
23,57,"[6, 8, 9, 11, 13, 16, 20, 21, 22, 24, 27, 28, 32]",56,35,0.799637


In [107]:
tm_20171031_NY_htopics[['Parent_ID','Topics','Child_Left_ID','Child_Right_ID','Distance']]

,Parent_ID,Topics,Child_Left_ID,Child_Right_ID,Distance
38,78,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",50,77,1.831349
37,77,"[0, 1, 2, 4, 7, 8, 9, 10, 11, 12, 13, 14, 15, ...",51,76,1.366483
36,76,"[0, 2, 4, 7, 8, 10, 11, 12, 13, 15, 17, 18, 19...",44,75,1.184720
35,75,"[0, 2, 4, 7, 8, 10, 11, 12, 13, 15, 17, 18, 19...",74,71,1.129344
34,74,"[2, 4, 7, 8, 10, 11, 12, 13, 15, 17, 18, 19, 2...",73,72,1.098938
33,73,"[2, 4, 11, 23, 24, 26, 29, 36, 37, 39]",67,69,1.073169
32,72,"[7, 8, 10, 12, 13, 15, 17, 18, 19, 21, 22, 30,...",66,70,1.019756
31,71,"[0, 28]",28,0,1.000000
30,70,"[8, 10, 13, 15, 18, 32, 35]",47,63,0.975919
29,69,"[2, 4, 24, 37, 39]",55,68,0.974006


In [ ]:
tm_20140524_CA_info = tm_20140524_CA.get_topic_info()
# Example: distance between topic 0 and topic 1
tm_20140524_CA_coordinates = tm_20140524_CA.topic_embeddings_  # shape: (n_topics, 2)
# Step 2: Compute the pairwise Euclidean distances
tm_20140524_CA_matrix = squareform(pdist(tm_20140524_CA_coordinates, metric='euclidean'))
# Step 3: Create topic labels
tm_20140524_CA_labels = [f"Topic {i}" for i in range(-1,len(tm_20140524_CA_coordinates)-1)]
# Step 4: Create a DataFrame for better formatting
tm_20140524_CA_dist_df = pd.DataFrame(tm_20140524_CA_matrix, index=tm_20140524_CA_labels, columns=tm_20140524_CA_labels)
# Step 5: Print the full matrix

tm_20250602_CO_info = tm_20250602_CO.get_topic_info()
# Example: distance between topic 0 and topic 1
tm_20250602_CO_coordinates = tm_20250602_CO.topic_embeddings_  # shape: (n_topics, 2)
# Step 2: Compute the pairwise Euclidean distances
tm_20250602_CO_matrix = squareform(pdist(tm_20250602_CO_coordinates, metric='euclidean'))
# Step 3: Create topic labels
tm_20250602_CO_labels = [f"Topic {i}" for i in range(-1,len(tm_20250602_CO_coordinates)-1)]
# Step 4: Create a DataFrame for better formatting
tm_20250602_CO_dist_df = pd.DataFrame(tm_20250602_CO_matrix, index=tm_20250602_CO_labels, columns=tm_20250602_CO_labels)

tm_20160612_FL_info = tm_20160612_FL.get_topic_info()
# Example: distance between topic 0 and topic 1
tm_20160612_FL_coordinates = tm_20160612_FL.topic_embeddings_  # shape: (n_topics, 2)
# Step 2: Compute the pairwise Euclidean distances
tm_20160612_FL_matrix = squareform(pdist(tm_20160612_FL_coordinates, metric='euclidean'))
# Step 3: Create topic labels
tm_20160612_FL_labels = [f"Topic {i}" for i in range(-1,len(tm_20160612_FL_coordinates)-1)]
# Step 4: Create a DataFrame for better formatting
tm_20160612_FL_dist_df = pd.DataFrame(tm_20160612_FL_matrix, index=tm_20160612_FL_labels, columns=tm_20160612_FL_labels)

tm_20171031_NY_info = tm_20171031_NY.get_topic_info()
# Example: distance between topic 0 and topic 1
tm_20171031_NY_coordinates = tm_20171031_NY.topic_embeddings_  # shape: (n_topics, 2)
# Step 2: Compute the pairwise Euclidean distances
tm_20171031_NY_matrix = squareform(pdist(tm_20171031_NY_coordinates, metric='euclidean'))
# Step 3: Create topic labels
tm_20171031_NY_labels = [f"Topic {i}" for i in range(-1,len(tm_20171031_NY_coordinates)-1)]
# Step 4: Create a DataFrame for better formatting
tm_20171031_NY_dist_df = pd.DataFrame(tm_20171031_NY_matrix, index=tm_20171031_NY_labels, columns=tm_20171031_NY_labels)

tm_20170812_VA_info = tm_20170812_VA.get_topic_info()
# Example: distance between topic 0 and topic 1
tm_20170812_VA_coordinates = tm_20170812_VA.topic_embeddings_  # shape: (n_topics, 2)
# Step 2: Compute the pairwise Euclidean distances
tm_20170812_VA_matrix = squareform(pdist(tm_20170812_VA_coordinates, metric='euclidean'))
# Step 3: Create topic labels
tm_20170812_VA_labels = [f"Topic {i}" for i in range(-1,len(tm_20170812_VA_coordinates)-1)]
# Step 4: Create a DataFrame for better formatting
tm_20170812_VA_dist_df = pd.DataFrame(tm_20170812_VA_matrix, index=tm_20170812_VA_labels, columns=tm_20170812_VA_labels)

In [77]:
pd.reset_option("all")


As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead.

: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.



In [119]:
tm_20140524_CA_info['Representative_Docs']


0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [117]:
f_20140524_CA

,GlobalEventID,date,Year,IsRootEvent,EventCode,EventBaseCode,EventRootCode,QuadClass,GoldsteinScale,NumMentions,...,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL,domain,target,title,text,description,factuality
0,298548627,20140517,2014,1,12,12,1,1,-0.4,2,...,-119.746,CA,20140524,http://www.cbsnews.com/news/elliot-rodger-cali...,www.cbsnews.com,cbsnews,Shooting spree suspect son of longtime Hollywo...,"Elliot Rodger, the suspect in Friday's deadly ...","A life of ""sadness, anger and hatred,"" accordi...",HIGH
1,298551064,20140524,2014,1,173,173,17,4,-5.0,3,...,-119.746,CA,20140524,http://news.yahoo.com/farmers-death-still-myst...,news.yahoo.com,news,"Yahoo News: Latest and Breaking News, Headline...",NaN,The latest news and headlines from Yahoo News....,HIGH
2,298551970,20140524,2014,1,193,193,19,4,-10.0,6,...,-119.746,CA,20140524,http://www.thesundaytimes.co.uk/sto/news/irela...,www.thetimes.com,thetimes,Latest news & breaking headlines,NaN,"The latest breaking UK, US, world, business an...",None
3,298554304,20140524,2014,1,190,190,19,4,-10.0,46,...,-119.746,CA,20140524,http://www.military.com/daily-news/2014/05/22/...,www.military.com,military,Military Daily News,NaN,Daily updates of everything that you need know...,HIGH
4,298556473,20140524,2014,1,190,190,19,4,-10.0,116,...,-119.746,CA,20140524,http://www.dw.de/california-gunman-was-hollywo...,www.dw.com,dw,US gunman 'was director's son' – DW – 05/25/2014,A Hollywood director says he believes his son ...,A Hollywood director says he believes his son ...,HIGH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,298634860,20140524,2014,1,193,193,19,4,-10.0,5,...,-119.746,CA,20140524,http://zeenews.india.com/news/world/seven-dead...,zeenews.india.com,zeenews,"Seven dead, several injured in California coll...",Los Angeles: Seven people were killed and seve...,Seven people were killed and seven others woun...,None
78,298635073,20140524,2014,0,190,190,19,4,-10.0,1,...,-119.746,CA,20140524,http://www.ibtimes.com/uc-santa-barbara-shooti...,www.ibtimes.com,ibtimes,UC Santa Barbara Shooting Update: 3 Stabbing V...,"Update 9:20 p.m. EDT: Elliot Rodger, the alleg...",The gunman allegedly began his murderous spree...,MIXED
79,298635186,20140524,2014,0,10,10,1,1,0.0,20,...,-119.746,CA,20140524,http://www.odt.co.nz/news/world/303498/six-kil...,www.odt.co.nz,odt,"'Madman' killer stabs three, shoots three",A gunman described as severely mentally distur...,A gunman described as severely mentally distur...,None
80,298636264,20140524,2014,1,10,10,1,1,0.0,6,...,-119.746,CA,20140524,http://time.com/112292/states-running-out-of-w...,time.com,time,These 7 States Are Running Out of Water,By Alexander E.M. Hess and Thomas C. Frohlich\...,This post is in partnership with 24/7Wall Stre...,HIGH
